In [1]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
from tkinter import Tk
from tkinter.filedialog import askopenfilename

def dummy(value):
    pass

def select_image():
    Tk().withdraw()
    filename = askopenfilename()
    return filename

def create_kernels():
    identity_kernel = np.array([[0,0,0],[0,1,0],[0,0,0]])
    sharpen_kernel = np.array([[0,-1,0], [-1,5,-1],[0, -1,0]])
    gaussian_kernel1 = np.outer(cv2.getGaussianKernel(3, 0), cv2.getGaussianKernel(3, 0).T)
    gaussian_kernel2 = np.outer(cv2.getGaussianKernel(5, 0), cv2.getGaussianKernel(5, 0).T)
    box_kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]], np.float32) / 9.0
    return [("Identity", identity_kernel), ("Sharpen", sharpen_kernel), 
            ("Gaussian 3x3", gaussian_kernel1), ("Gaussian 5x5", gaussian_kernel2), 
            ("Box Blur", box_kernel)]

def change_color(image, r, g, b):
    image = image.astype(np.float32)
    image[:,:,0] = np.clip(image[:,:,0] * b / 100, 0, 255)
    image[:,:,1] = np.clip(image[:,:,1] * g / 100, 0, 255)
    image[:,:,2] = np.clip(image[:,:,2] * r / 100, 0, 255)
    return image.astype(np.uint8)

def setup_ui(kernel_names):
    cv2.namedWindow('app')
    cv2.createTrackbar('contrast', 'app', 1, 100, dummy)
    cv2.createTrackbar('brightness', 'app', 50, 100, dummy)
    for i, (name, _) in enumerate(kernel_names):
        cv2.createTrackbar(name, 'app', 0, 1, dummy)
    cv2.createTrackbar('grayscale', 'app', 0, 100, dummy)
    cv2.createTrackbar('R', 'app', 100, 200, dummy)
    cv2.createTrackbar('G', 'app', 100, 200, dummy)
    cv2.createTrackbar('B', 'app', 100, 200, dummy)

def main():
    kernels = create_kernels()
    image_path = select_image()
    if not image_path:
        print("No file selected")
        return

    color_original = cv2.imread(image_path)
    if color_original is None:
        print("Error loading image")
        return

    # Convert to grayscale and then back to BGR to maintain consistent channels
    gray_original = cv2.cvtColor(color_original, cv2.COLOR_BGR2GRAY)
    gray_original = cv2.cvtColor(gray_original, cv2.COLOR_GRAY2BGR)

    setup_ui(kernels)
    count = 1

    while True:
        try:
            grayscale_level = cv2.getTrackbarPos('grayscale', 'app')
            contrast = cv2.getTrackbarPos('contrast', 'app')
            brightness = cv2.getTrackbarPos('brightness', 'app')
            r = cv2.getTrackbarPos('R', 'app')
            g = cv2.getTrackbarPos('G', 'app')
            b = cv2.getTrackbarPos('B', 'app')

            # Start with the original images for modifications
            color_modified = color_original.copy()
            gray_modified = gray_original.copy()

            # Apply each enabled filter
            for i, (_, kernel) in enumerate(kernels):
                if cv2.getTrackbarPos(kernels[i][0], 'app'):
                    color_modified = cv2.filter2D(color_modified, -1, kernel)
                    gray_modified = cv2.filter2D(gray_modified, -1, kernel)

            # Apply color change
            color_modified = change_color(color_modified, r, g, b)
            gray_modified = change_color(gray_modified, r, g, b)

            # Map contrast and apply brightness
            mapped_contrast = 0.1 + (contrast / 100) * 1.9
            color_modified = cv2.addWeighted(color_modified, mapped_contrast, np.zeros_like(color_original), 0, brightness - 50)
            gray_modified = cv2.addWeighted(gray_modified, mapped_contrast, np.zeros_like(gray_original), 0, brightness - 50)

            # Blend color and grayscale images based on grayscale level
            blended_image = cv2.addWeighted(color_modified, (100 - grayscale_level) / 100, gray_modified, grayscale_level / 100, 0)

            # Display the blended image
            cv2.imshow('app', blended_image)

            key = cv2.waitKey(100)
            if key == ord('q'):
                break
            elif key == ord('s'):
                cv2.imwrite(f'output-{count}.jpg', blended_image)
                count += 1
        except Exception as e:
            print("An error occurred:", e)
            break

    # Close the OpenCV window and release resources
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()
